## Hamiltonian reduction

In [6]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end


## Use NOON to remove terms for UCCSD operators


In [7]:
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()

NOON_spins_combined

array([1.99991509e+00, 1.96744892e+00, 2.71958861e-02, 7.94453898e-05,
       2.68032899e-03, 2.68032899e-03])

The LiH ground state under JW is:

$$|\psi \rangle_{HF}^{ground} = | 1 1 1 1 \:  0 0 0 0 0 0 0 0 \rangle$$

and under BK is:

$$|\psi \rangle_{HF}^{ground} = | 1 0 1 0 0 0 0 0 0 0 0 0 \rangle$$

Looking at the NOON we see that orbitals with indices:
- 0,1 remain OCCUPIED
- 6,7 remain UN_occupied


In [3]:
from quchem.Ansatz_Generator_Functions import *

ansatz_obj = BK_Qubit_Reduction(QubitHamiltonian,
                             Hamilt.molecule.n_electrons,
                             Hamilt.molecule.n_qubits)


#### Use this to reduce Hamiltonian:

In [8]:
list_of_qubit_indices_to_remove = [0,1,6,7]
reduced_Qubit_Hamiltonian = ansatz_obj.Remove_indices_from_Hamiltonian(list_of_qubit_indices_to_remove)
print('Hamiltonian size reduced: {} --> {}'.format(len(list(QubitHamiltonian)), len(list(reduced_Qubit_Hamiltonian))))

Hamiltonian size reduced: 631 --> 450


#### Re_label Hamiltonian:

In [9]:
qubit_re_label_dict, reduced_RE_LABELLED_Qubit_Hamiltonian = ansatz_obj.Re_label_Hamiltonian(reduced_Qubit_Hamiltonian)
reduced_RE_LABELLED_Qubit_Hamiltonian

(-6.030955269133994+0j) [] +
(-0.05811838633732249+0j) [X0] +
(0.009702964590343754+0j) [X0 X1 X2 X3] +
(-0.0012091978607673888+0j) [X0 X1 X2 Y7] +
(-0.0016354065601825055+0j) [X0 X1 Y2 X3] +
(-0.00020377653620312224+0j) [X0 X1 Y2 Y3] +
(0.0012091978607673888+0j) [X0 X1 Y2 Z3 Z5 Z6 X7] +
(0.0016354065601825057+0j) [X0 X1 Z2 X3] +
(0.0048895396515473664+0j) [X0 X1 Z2 Y3] +
(-0.00384648213715215+0j) [X0 X1 Z2 Z3 Z5 Y6 X7] +
(-0.0026372842763847586+0j) [X0 X1 Z2 Z5 Y6 X7] +
(0.004031651676556207+0j) [X0 X1 X4 Y7] +
(-0.004031651676556207+0j) [X0 X1 Y4 Z6 X7] +
(-0.00976749332732197+0j) [X0 X1 Z4 Z5 Y6 X7] +
(-0.005735841650765762+0j) [X0 X1 Z4 Y6 X7] +
(-0.003136942278337769+0j) [X0 X1 Z5 X6 X7] +
(0.02666913300964253+0j) [X0 X1 Z5 Y6 X7] +
(-0.0027468618195562876+0j) [X0 X1 Z5 Z6 X7] +
(-0.03893002781929342+0j) [X0 X1 X6 Y7] +
(0.00442173213533769+0j) [X0 X1 Y7] +
(0.0016354065601825055+0j) [X0 Y1 X2 X3] +
(0.00034734865840738843+0j) [X0 Y1 X2 Y3] +
(-0.005741602942963053+0j) [X0 Y1 Y2 X

#### Find new FCI energy

In [10]:
new_Molecular_H_MATRIX = Hamilt.Get_sparse_Qubit_Hamiltonian_matrix(reduced_RE_LABELLED_Qubit_Hamiltonian)

from scipy.sparse.linalg import eigs
eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
new_FCI_Energy = min(eig_values)

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', Hamilt.molecule.fci_energy)

new_FCI =  (-7.827946663038102-1.1291153760637219e-16j) VS old FCI: -7.784460280267057


#### simplify Ansatz

In [13]:
# automate:
# reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
#     NOON=NOON_spins_combined,
#     occ_threshold= 1.999,
#     unocc_threshold=1e-4,
#     indices_to_remove_list_manual=None, 
#     single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
#     double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
#     singles_hamiltonian=Hamilt.singles_hamiltonian,
#     doubles_hamiltonian=Hamilt.doubles_hamiltonian,
#     tol_filter_small_terms=None)

# manual
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    indices_to_remove_list_manual=[0,1,6,7])

In [14]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()
print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

REDUCTION
ia_terms 16 TO 6
ijab_terms 42 TO 6


In [21]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab)

reduced_CC_ijab = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ijab, list_of_qubit_indices_to_remove)


# RELABEL qubits!
reduced_RE_LABELLED_CC_ijab = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ijab)
reduced_RE_LABELLED_CC_ijab

In [22]:
reduced_CC_ia = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ia, list_of_qubit_indices_to_remove)

reduced_RE_LABELLED_CC_ia = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ia)
reduced_RE_LABELLED_CC_ia

[-0.5j [X0 Z1 Y2] +
 0.5j [Y0 Z1 X2],
 -0.5j [X0 Z1 Z2 Z3 Y4] +
 0.5j [Y0 Z1 Z2 Z3 X4],
 -0.5j [X0 Z1 Z2 Z3 Z4 Z5 Y6] +
 0.5j [Y0 Z1 Z2 Z3 Z4 Z5 X6],
 -0.5j [X1 Z2 Y3] +
 0.5j [Y1 Z2 X3],
 -0.5j [X1 Z2 Z3 Z4 Y5] +
 0.5j [Y1 Z2 Z3 Z4 X5],
 -0.5j [X1 Z2 Z3 Z4 Z5 Z6 Y7] +
 0.5j [Y1 Z2 Z3 Z4 Z5 Z6 X7]]

#### Get reduced HF input state

In [16]:
print('old input = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())
print('BUT following indices removed:', list_of_qubit_indices_to_remove)
ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)

old input =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
BUT following indices removed: [0, 1, 6, 7]


array([1., 0., 0., 0., 0., 0., 0., 0.])

# Ansatz Q circuit

In [23]:
N_QUBITS = Hamilt.molecule.n_qubits - len(list_of_qubit_indices_to_remove)
# or len(ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove))

N_electrons = int(sum(ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)))

full_ansatz_Q_Circ = Ansatz_Circuit(reduced_RE_LABELLED_CC_ia, reduced_RE_LABELLED_CC_ijab,
             N_QUBITS, N_electrons, manual_HF_state=ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove))

ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(reduced_theta_parameters_ia, 
                                                              reduced_theta_parameters_ijab,
                                                              transformation='BK')

print(ansatz_cirq_circuit.to_text_diagram(transpose=True)) 

0         1         2         3         4         5         6         7
│         │         │         │         │         │         │         │
X         │         H         │         H         │         H         H
│         │         │         │         │         │         │         │
Rx(0.5π)  │         │         │         │         │         │         │
│         │         │         │         │         │         │         │
@─────────X         │         │         │         │         │         │
│         │         │         │         │         │         │         │
│         @─────────X         │         │         │         │         │
│         │         │         │         │         │         │         │
│         │         Rz(-0.0π) │         │         │         │         │
│         │         │         │         │         │         │         │
│         @─────────X         │         │         │         │         │
│         │         │         │         │         │         │   